In [28]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [98]:
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.Many years later, the village was holding a hunting competition. The girl's father, the priest, the prince, and the advisor were all there. The girl recognized them (knew who they were), but they did not recognize her.

At the competition, the girl stood up and told her life story to the crowd.

Everyone immediately recognized her. They understood she was telling the truth! The girl, the prince, and her father were all happily reunited (brought together again). The priest and advisor were banished to prison.

They soon learned that the son who had been stolen was actually living in the village. The girl and her husband, the prince, were then reunited with their son.

The prince was so happy that he passed a law that said that hunters could not hunt (kill) gazelles. From that day on, the gazelles were protected from harm.
""".split()


vocab=set(raw_text)
vocab_size=len(vocab)
word2idx={word:i for i,word in enumerate(vocab)}
#print(word2idx)
embedding_size=6


data=[]
for i in range(2,len(raw_text)-2):
    context=[raw_text[i - 2], raw_text[i - 1],raw_text[i + 1], raw_text[i + 2]]
    target=raw_text[i]
    data.append((context,target))

print(data[0:6])

def make_context_vector(context,word2idx):
    idxs=[word2idx[w] for w in context]
    return torch.tensor(idxs,dtype=torch.long)
def make_context_vector1(context,word2idx):
    idxs=word2idx[context]
    return torch.tensor(idxs,dtype=torch.long)


a=make_context_vector(data[0][0],word2idx)
print (data[0][1]) 


e=nn.Embedding(vocab_size,embedding_size)
a=e(a)
#print(a.size())


[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea'), (['the', 'idea', 'a', 'computational'], 'of')]
about


In [118]:







class LSTM(nn.Module):
    def __init__(self,input_size,hidden_size,batch_size,output_size,num_layers):
        super(LSTM,self).__init__()
        
        self.hidden_size=hidden_size
        
        self.output_size=output_size
        self.input_size=input_size
        self.num_layers=num_layers
        self.lstm=nn.LSTM(self.input_size,self.hidden_size,self.num_layers)
        
        self.word_embeddings=nn.Embedding(vocab_size,embedding_size)
        
        self.linear=nn.Linear(self.hidden_size,embedding_size)
        
        
        
        
        
    def forward(self,inputi,target):
        
        embedsi=self.word_embeddings(inputi)
        
        embedso=self.word_embeddings(target)
        
        out,self.hidden=self.lstm(embedsi.view(len(inputi),1,-1))
        #out=torch.randn(1,hidden_size)
        
        y_pred = self.linear(out[-1].view(1, -1))
    
        loss=((embedso-y_pred)**2).mean()
    
        return loss
        


In [119]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=1):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.word_embeddings=nn.Embedding(vocab_size,embedding_size)
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, inputi,target):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        embedsi=self.word_embeddings(inputi)
        
        embedso=self.word_embeddings(target)
        out,self.hidden=self.lstm(embedsi.view(len(inputi),1,-1))
        #out=torch.randn(1,hidden_size)
        
        y_pred = self.linear(out[-1].view(1, -1))
    
        loss=((embedso-y_pred)**2).mean()
        
        return loss

In [120]:
epochs=50
model=LSTM(6,6,1,6,1)
optimiser=torch.optim.Adam(model.parameters(),lr=1e-6)
with torch.no_grad():
    inputii=make_context_vector(data[0][0],word2idx)
    target=make_context_vector1(data[0][1],word2idx)
    print(target)
    loss=model(inputii,target)
    print(loss)

for epoch in range (1,epochs):
    for a,b in data:
        model.zero_grad
        inputii=make_context_vector(a,word2idx)
        targett=make_context_vector1(b,word2idx)
        loss=model(inputii,targett)
        loss.backward()
        optimiser.step()
        print(loss)
    
    
    
    
    
    
    
    
    
    


tensor(134)
tensor(1.1147)
tensor(1.1147, grad_fn=<MeanBackward0>)
tensor(0.5468, grad_fn=<MeanBackward0>)
tensor(0.9105, grad_fn=<MeanBackward0>)
tensor(0.1788, grad_fn=<MeanBackward0>)
tensor(1.2304, grad_fn=<MeanBackward0>)
tensor(0.6858, grad_fn=<MeanBackward0>)
tensor(1.6144, grad_fn=<MeanBackward0>)
tensor(0.8843, grad_fn=<MeanBackward0>)
tensor(1.3550, grad_fn=<MeanBackward0>)
tensor(0.7003, grad_fn=<MeanBackward0>)
tensor(0.7044, grad_fn=<MeanBackward0>)
tensor(0.2883, grad_fn=<MeanBackward0>)
tensor(0.6580, grad_fn=<MeanBackward0>)
tensor(2.1020, grad_fn=<MeanBackward0>)
tensor(0.2554, grad_fn=<MeanBackward0>)
tensor(2.1244, grad_fn=<MeanBackward0>)
tensor(0.9854, grad_fn=<MeanBackward0>)
tensor(1.5746, grad_fn=<MeanBackward0>)
tensor(1.4783, grad_fn=<MeanBackward0>)
tensor(0.4566, grad_fn=<MeanBackward0>)
tensor(0.7352, grad_fn=<MeanBackward0>)
tensor(0.8676, grad_fn=<MeanBackward0>)
tensor(0.7333, grad_fn=<MeanBackward0>)
tensor(0.6919, grad_fn=<MeanBackward0>)
tensor(0.7581

tensor(1.6832, grad_fn=<MeanBackward0>)
tensor(0.3565, grad_fn=<MeanBackward0>)
tensor(0.2928, grad_fn=<MeanBackward0>)
tensor(0.2843, grad_fn=<MeanBackward0>)
tensor(0.5348, grad_fn=<MeanBackward0>)
tensor(1.7177, grad_fn=<MeanBackward0>)
tensor(1.7738, grad_fn=<MeanBackward0>)
tensor(0.7174, grad_fn=<MeanBackward0>)
tensor(1.6218, grad_fn=<MeanBackward0>)
tensor(1.4073, grad_fn=<MeanBackward0>)
tensor(1.5816, grad_fn=<MeanBackward0>)
tensor(0.5424, grad_fn=<MeanBackward0>)
tensor(0.9610, grad_fn=<MeanBackward0>)
tensor(1.1381, grad_fn=<MeanBackward0>)
tensor(0.4346, grad_fn=<MeanBackward0>)
tensor(0.5237, grad_fn=<MeanBackward0>)
tensor(0.8571, grad_fn=<MeanBackward0>)
tensor(0.4327, grad_fn=<MeanBackward0>)
tensor(1.2546, grad_fn=<MeanBackward0>)
tensor(0.8404, grad_fn=<MeanBackward0>)
tensor(1.4416, grad_fn=<MeanBackward0>)
tensor(0.6344, grad_fn=<MeanBackward0>)
tensor(0.1685, grad_fn=<MeanBackward0>)
tensor(0.7643, grad_fn=<MeanBackward0>)
tensor(0.7218, grad_fn=<MeanBackward0>)


tensor(1.3981, grad_fn=<MeanBackward0>)
tensor(1.4058, grad_fn=<MeanBackward0>)
tensor(2.5627, grad_fn=<MeanBackward0>)
tensor(0.4973, grad_fn=<MeanBackward0>)
tensor(0.9864, grad_fn=<MeanBackward0>)
tensor(1.1593, grad_fn=<MeanBackward0>)
tensor(1.1114, grad_fn=<MeanBackward0>)
tensor(1.6435, grad_fn=<MeanBackward0>)
tensor(1.4496, grad_fn=<MeanBackward0>)
tensor(0.2593, grad_fn=<MeanBackward0>)
tensor(0.6937, grad_fn=<MeanBackward0>)
tensor(1.4233, grad_fn=<MeanBackward0>)
tensor(0.4461, grad_fn=<MeanBackward0>)
tensor(2.0254, grad_fn=<MeanBackward0>)
tensor(1.7785, grad_fn=<MeanBackward0>)
tensor(4.1335, grad_fn=<MeanBackward0>)
tensor(0.3107, grad_fn=<MeanBackward0>)
tensor(0.1828, grad_fn=<MeanBackward0>)
tensor(4.1910, grad_fn=<MeanBackward0>)
tensor(0.1641, grad_fn=<MeanBackward0>)
tensor(0.5595, grad_fn=<MeanBackward0>)
tensor(0.5363, grad_fn=<MeanBackward0>)
tensor(0.6235, grad_fn=<MeanBackward0>)
tensor(0.3966, grad_fn=<MeanBackward0>)
tensor(0.7543, grad_fn=<MeanBackward0>)


tensor(1.3510, grad_fn=<MeanBackward0>)
tensor(0.5769, grad_fn=<MeanBackward0>)
tensor(0.4078, grad_fn=<MeanBackward0>)
tensor(0.7017, grad_fn=<MeanBackward0>)
tensor(1.2169, grad_fn=<MeanBackward0>)
tensor(2.5182, grad_fn=<MeanBackward0>)
tensor(1.0383, grad_fn=<MeanBackward0>)
tensor(0.3663, grad_fn=<MeanBackward0>)
tensor(1.3439, grad_fn=<MeanBackward0>)
tensor(1.4096, grad_fn=<MeanBackward0>)
tensor(0.9339, grad_fn=<MeanBackward0>)
tensor(0.4552, grad_fn=<MeanBackward0>)
tensor(1.7464, grad_fn=<MeanBackward0>)
tensor(0.8209, grad_fn=<MeanBackward0>)
tensor(1.5293, grad_fn=<MeanBackward0>)
tensor(0.2923, grad_fn=<MeanBackward0>)
tensor(0.3672, grad_fn=<MeanBackward0>)
tensor(0.1631, grad_fn=<MeanBackward0>)
tensor(1.1502, grad_fn=<MeanBackward0>)
tensor(1.6490, grad_fn=<MeanBackward0>)
tensor(1.0989, grad_fn=<MeanBackward0>)
tensor(0.6503, grad_fn=<MeanBackward0>)
tensor(2.4443, grad_fn=<MeanBackward0>)
tensor(1.2002, grad_fn=<MeanBackward0>)
tensor(1.7340, grad_fn=<MeanBackward0>)


tensor(1.4180, grad_fn=<MeanBackward0>)
tensor(1.2349, grad_fn=<MeanBackward0>)
tensor(0.5820, grad_fn=<MeanBackward0>)
tensor(0.5995, grad_fn=<MeanBackward0>)
tensor(1.5242, grad_fn=<MeanBackward0>)
tensor(1.3272, grad_fn=<MeanBackward0>)
tensor(2.4367, grad_fn=<MeanBackward0>)
tensor(0.9098, grad_fn=<MeanBackward0>)
tensor(1.0452, grad_fn=<MeanBackward0>)
tensor(1.8329, grad_fn=<MeanBackward0>)
tensor(1.2946, grad_fn=<MeanBackward0>)
tensor(0.3312, grad_fn=<MeanBackward0>)
tensor(0.1568, grad_fn=<MeanBackward0>)
tensor(1.0603, grad_fn=<MeanBackward0>)
tensor(1.6788, grad_fn=<MeanBackward0>)
tensor(1.7274, grad_fn=<MeanBackward0>)
tensor(0.3608, grad_fn=<MeanBackward0>)
tensor(1.6644, grad_fn=<MeanBackward0>)
tensor(0.4065, grad_fn=<MeanBackward0>)
tensor(0.7392, grad_fn=<MeanBackward0>)
tensor(0.4928, grad_fn=<MeanBackward0>)
tensor(1.6074, grad_fn=<MeanBackward0>)
tensor(0.0958, grad_fn=<MeanBackward0>)
tensor(0.6234, grad_fn=<MeanBackward0>)
tensor(1.3115, grad_fn=<MeanBackward0>)


...

tensor(1.9759, grad_fn=<MeanBackward0>)
tensor(1.7281, grad_fn=<MeanBackward0>)
tensor(4.0346, grad_fn=<MeanBackward0>)
tensor(0.2853, grad_fn=<MeanBackward0>)
tensor(0.1702, grad_fn=<MeanBackward0>)
tensor(4.1373, grad_fn=<MeanBackward0>)
tensor(0.1550, grad_fn=<MeanBackward0>)
tensor(0.5174, grad_fn=<MeanBackward0>)
tensor(0.5209, grad_fn=<MeanBackward0>)
tensor(0.5933, grad_fn=<MeanBackward0>)
tensor(0.3638, grad_fn=<MeanBackward0>)
tensor(0.7331, grad_fn=<MeanBackward0>)
tensor(0.4495, grad_fn=<MeanBackward0>)
tensor(0.4171, grad_fn=<MeanBackward0>)
tensor(0.5658, grad_fn=<MeanBackward0>)
tensor(0.5607, grad_fn=<MeanBackward0>)
tensor(0.1511, grad_fn=<MeanBackward0>)
tensor(1.7960, grad_fn=<MeanBackward0>)
tensor(0.6416, grad_fn=<MeanBackward0>)
tensor(1.6060, grad_fn=<MeanBackward0>)
tensor(0.9119, grad_fn=<MeanBackward0>)
tensor(4.0843, grad_fn=<MeanBackward0>)
tensor(0.9423, grad_fn=<MeanBackward0>)
tensor(2.1091, grad_fn=<MeanBackward0>)
tensor(1.5171, grad_fn=<MeanBackward0>)


tensor(0.7169, grad_fn=<MeanBackward0>)
tensor(1.3261, grad_fn=<MeanBackward0>)
tensor(1.3062, grad_fn=<MeanBackward0>)
tensor(1.2815, grad_fn=<MeanBackward0>)
tensor(0.5438, grad_fn=<MeanBackward0>)
tensor(0.3939, grad_fn=<MeanBackward0>)
tensor(0.6733, grad_fn=<MeanBackward0>)
tensor(1.1845, grad_fn=<MeanBackward0>)
tensor(2.4505, grad_fn=<MeanBackward0>)
tensor(1.0120, grad_fn=<MeanBackward0>)
tensor(0.3396, grad_fn=<MeanBackward0>)
tensor(1.3040, grad_fn=<MeanBackward0>)
tensor(1.3613, grad_fn=<MeanBackward0>)
tensor(0.8760, grad_fn=<MeanBackward0>)
tensor(0.4494, grad_fn=<MeanBackward0>)
tensor(1.7018, grad_fn=<MeanBackward0>)
tensor(0.8001, grad_fn=<MeanBackward0>)
tensor(1.4854, grad_fn=<MeanBackward0>)
tensor(0.2798, grad_fn=<MeanBackward0>)
tensor(0.3576, grad_fn=<MeanBackward0>)
tensor(0.1549, grad_fn=<MeanBackward0>)
tensor(1.1548, grad_fn=<MeanBackward0>)
tensor(1.6134, grad_fn=<MeanBackward0>)
tensor(1.0714, grad_fn=<MeanBackward0>)
tensor(0.6365, grad_fn=<MeanBackward0>)


tensor(0.4624, grad_fn=<MeanBackward0>)
tensor(0.3252, grad_fn=<MeanBackward0>)
tensor(0.3834, grad_fn=<MeanBackward0>)
tensor(1.5884, grad_fn=<MeanBackward0>)
tensor(0.1455, grad_fn=<MeanBackward0>)
tensor(1.4096, grad_fn=<MeanBackward0>)
tensor(1.3620, grad_fn=<MeanBackward0>)
tensor(1.2189, grad_fn=<MeanBackward0>)
tensor(0.5478, grad_fn=<MeanBackward0>)
tensor(0.5735, grad_fn=<MeanBackward0>)
tensor(1.5091, grad_fn=<MeanBackward0>)
tensor(1.2957, grad_fn=<MeanBackward0>)
tensor(2.3707, grad_fn=<MeanBackward0>)
tensor(0.8692, grad_fn=<MeanBackward0>)
tensor(1.0419, grad_fn=<MeanBackward0>)
tensor(1.7915, grad_fn=<MeanBackward0>)
tensor(1.2770, grad_fn=<MeanBackward0>)
tensor(0.3255, grad_fn=<MeanBackward0>)
tensor(0.1455, grad_fn=<MeanBackward0>)
tensor(1.0312, grad_fn=<MeanBackward0>)
tensor(1.6564, grad_fn=<MeanBackward0>)
tensor(1.7012, grad_fn=<MeanBackward0>)
tensor(0.3536, grad_fn=<MeanBackward0>)
tensor(1.6330, grad_fn=<MeanBackward0>)
tensor(0.3938, grad_fn=<MeanBackward0>)


tensor(0.1477, grad_fn=<MeanBackward0>)
tensor(0.9357, grad_fn=<MeanBackward0>)
tensor(1.3252, grad_fn=<MeanBackward0>)
tensor(0.6803, grad_fn=<MeanBackward0>)
tensor(1.1228, grad_fn=<MeanBackward0>)
tensor(0.5379, grad_fn=<MeanBackward0>)
tensor(0.8756, grad_fn=<MeanBackward0>)
tensor(0.1672, grad_fn=<MeanBackward0>)
tensor(1.1722, grad_fn=<MeanBackward0>)
tensor(0.6652, grad_fn=<MeanBackward0>)
tensor(1.5916, grad_fn=<MeanBackward0>)
tensor(0.8409, grad_fn=<MeanBackward0>)
tensor(1.3576, grad_fn=<MeanBackward0>)
tensor(0.6782, grad_fn=<MeanBackward0>)
tensor(0.6886, grad_fn=<MeanBackward0>)
tensor(0.2747, grad_fn=<MeanBackward0>)
tensor(0.6528, grad_fn=<MeanBackward0>)
tensor(2.0252, grad_fn=<MeanBackward0>)
tensor(0.2530, grad_fn=<MeanBackward0>)
tensor(2.1183, grad_fn=<MeanBackward0>)
tensor(0.9400, grad_fn=<MeanBackward0>)
tensor(1.5868, grad_fn=<MeanBackward0>)
tensor(1.4270, grad_fn=<MeanBackward0>)
tensor(0.4428, grad_fn=<MeanBackward0>)
tensor(0.7176, grad_fn=<MeanBackward0>)


tensor(0.6022, grad_fn=<MeanBackward0>)
tensor(2.4283, grad_fn=<MeanBackward0>)
tensor(1.1943, grad_fn=<MeanBackward0>)
tensor(0.6804, grad_fn=<MeanBackward0>)
tensor(1.6489, grad_fn=<MeanBackward0>)
tensor(0.3407, grad_fn=<MeanBackward0>)
tensor(0.2786, grad_fn=<MeanBackward0>)
tensor(0.2675, grad_fn=<MeanBackward0>)
tensor(0.5105, grad_fn=<MeanBackward0>)
tensor(1.6823, grad_fn=<MeanBackward0>)
tensor(1.7386, grad_fn=<MeanBackward0>)
tensor(0.6951, grad_fn=<MeanBackward0>)
tensor(1.5941, grad_fn=<MeanBackward0>)
tensor(1.3746, grad_fn=<MeanBackward0>)
tensor(1.5714, grad_fn=<MeanBackward0>)
tensor(0.5408, grad_fn=<MeanBackward0>)
tensor(0.9543, grad_fn=<MeanBackward0>)
tensor(1.0780, grad_fn=<MeanBackward0>)
tensor(0.4097, grad_fn=<MeanBackward0>)
tensor(0.5143, grad_fn=<MeanBackward0>)
tensor(0.8102, grad_fn=<MeanBackward0>)
tensor(0.4039, grad_fn=<MeanBackward0>)
tensor(1.2059, grad_fn=<MeanBackward0>)
tensor(0.8263, grad_fn=<MeanBackward0>)
tensor(1.3990, grad_fn=<MeanBackward0>)


In [121]:
with torch.no_grad():
    inputii=make_context_vector(data[0][0],word2idx)
    target=make_context_vector1(data[0][1],word2idx)
    print(target)
    loss=model(inputii,target)
    print(loss)

tensor(134)
tensor(1.1231)
